## SOAR Data
This Jupyter notebook imports data about the SOAR facility.

There are two parts to this notebook.
1. Some simple code to read and decode the SOAR data (in JSON format);
2. Code that makes use of the VeRoViz package to visualize the SOAR data.

--- 

### 1.  Read JSON data

In [1]:
import json
import urllib.request

In [2]:
with urllib.request.urlopen("https://raw.githubusercontent.com/optimatorlab/SOAR/master/data/soar.json") as url:
    soar_data = json.loads(url.read().decode())

soar_data

{'polesArray': [{'id': 0,
   'lat': 42.99525466,
   'lon': -78.79684403,
   'elevMSLmeters': 180.43,
   'heightMeters': 25.908,
   'elevMSLcesiumMeters': 146.579},
  {'id': 1,
   'lat': 42.99534179,
   'lon': -78.79675314,
   'elevMSLmeters': 180.61,
   'heightMeters': 25.908,
   'elevMSLcesiumMeters': 146.57},
  {'id': 2,
   'lat': 42.99542893,
   'lon': -78.79666224,
   'elevMSLmeters': 180.69,
   'heightMeters': 25.908,
   'elevMSLcesiumMeters': 146.565},
  {'id': 3,
   'lat': 42.99551606,
   'lon': -78.79657134,
   'elevMSLmeters': 180.86,
   'heightMeters': 25.908,
   'elevMSLcesiumMeters': 146.56},
  {'id': 4,
   'lat': 42.99559946,
   'lon': -78.79671971,
   'elevMSLmeters': 181.16,
   'heightMeters': 25.908,
   'elevMSLcesiumMeters': 146.547},
  {'id': 5,
   'lat': 42.99568287,
   'lon': -78.79686809,
   'elevMSLmeters': 181.33,
   'heightMeters': 25.908,
   'elevMSLcesiumMeters': 146.533},
  {'id': 6,
   'lat': 42.99576627,
   'lon': -78.79701647,
   'elevMSLmeters': 181.45,
 

In [3]:
for item in soar_data['polesArray']:
    print('id:', item['id'], 
          'lat:', item['lat'], 
          'lon:', item['lon'],
          'elevMSLmeters:', item['elevMSLmeters'],
          'heightMeters:', item['heightMeters'])

id: 0 lat: 42.99525466 lon: -78.79684403 elevMSLmeters: 180.43 heightMeters: 25.908
id: 1 lat: 42.99534179 lon: -78.79675314 elevMSLmeters: 180.61 heightMeters: 25.908
id: 2 lat: 42.99542893 lon: -78.79666224 elevMSLmeters: 180.69 heightMeters: 25.908
id: 3 lat: 42.99551606 lon: -78.79657134 elevMSLmeters: 180.86 heightMeters: 25.908
id: 4 lat: 42.99559946 lon: -78.79671971 elevMSLmeters: 181.16 heightMeters: 25.908
id: 5 lat: 42.99568287 lon: -78.79686809 elevMSLmeters: 181.33 heightMeters: 25.908
id: 6 lat: 42.99576627 lon: -78.79701647 elevMSLmeters: 181.45 heightMeters: 25.908
id: 7 lat: 42.99584968 lon: -78.79716485 elevMSLmeters: 181.55 heightMeters: 25.908
id: 8 lat: 42.99576254 lon: -78.79725575 elevMSLmeters: 181.44 heightMeters: 25.908
id: 9 lat: 42.99567541 lon: -78.79734665 elevMSLmeters: 181.39 heightMeters: 25.908
id: 10 lat: 42.99558828 lon: -78.79743755 elevMSLmeters: 181.36 heightMeters: 25.908
id: 11 lat: 42.99550487 lon: -78.79728917 elevMSLmeters: 181.1 heightMeters

### 2. Visualize data on maps 

You'll need the `veroviz` package to continue.  To install:
```
pip install veroviz
```

In [4]:
import veroviz as vrv
vrv.checkVersion()

'Your current installed version of veroviz is 0.4.5. You are up-to-date with the latest available version.'

In [5]:
# Create a "nodes" dataframe from the `soar_data` object (imported in Step 1)
myNodes = vrv.initDataframe('nodes')

for item in soar_data['polesArray']:
    locs = [item['lat'], item['lon']]
    myNodes = vrv.createNodesFromLocs(
                    locs              = [locs], 
                    initNodes         = myNodes,
                    startNode         = item['id'],
                    nodeType          = 'Poles', 
                    nodeName          = 'pole%d' % item['id'], 
                    leafletIconPrefix = 'custom', 
                    leafletIconType   = '10-white-10', 
                    leafletIconText   = item['id'])

In [6]:
# Initialize map and draw blue poles
myMap = vrv.createLeaflet(nodes = myNodes, mapBackground = 'Arcgis Aerial', zoomStart = 18)
myMap

In [7]:
# Add curbs
for i in soar_data['curbsArray']:
    myMap = vrv.addLeafletPolygon(mapObject = myMap,
                              zoomStart     = 18,
                              points        = i['poly'], 
                              lineWeight    = 3, 
                              lineColor     = 'pink', 
                              lineOpacity   = 0.8, 
                              lineStyle     = 'solid',
                              fillColor     = 'pink', 
                              fillOpacity   = 0.5)
myMap

In [8]:
# Add light poles
for i in soar_data['lightsArray']:
    myMap = vrv.addLeafletCircle(mapObject   = myMap, 
                                 zoomStart   = 18,
                                 center      = [i['lat'], i['lon']], 
                                 radius      = i['radius'], 
                                 popupText   = 'lightpole %d' % i['id'], 
                                 lineWeight  = 3, 
                                 lineColor   = None, 
                                 lineOpacity = 0.8, 
                                 lineStyle   = 'solid', 
                                 fillColor   = 'red', 
                                 fillOpacity = 0.8)
myMap

In [9]:
# Add geofence
myMap = vrv.addLeafletPolygon(mapObject=myMap, 
                              zoomStart   = 18, 
                              points      = soar_data['geofence']['poly'], 
                              popupText   = None, 
                              lineWeight  = 3, 
                              lineColor   = 'red', 
                              lineOpacity = 0.8, 
                              lineStyle   = 'solid', 
                              fillColor   = None, 
                              fillOpacity = 0.3)
myMap

--- 

### 3.  Error-Checking

We need to bet better readings on our poles!

In [10]:
# Poles 0-1, 1-2, 2-3, 7-8, 8-9, 9-10 should be 40-feet apart.
print('Poles 0-1, 1-2, 2-3, 7-8, 8-9, 9-10 should be 40-feet apart.')
for (i,j) in [[0,1], [1,2], [2,3], [7,8], [8,9], [9,10]]:
    loc1 = list(myNodes[myNodes['id'] == i][['lat', 'lon']].values[0])
    loc2 = list(myNodes[myNodes['id'] == j][['lat', 'lon']].values[0])
    distMeters = vrv.distance2D(loc1, loc2)
    distFeet = vrv.convertDistance(distMeters, 'meters', 'feet')
    print('From Pole %d to %d: %f meters (%f feet)' % (i, j, distMeters, distFeet))


# Poles 3-4, 4-5, 5-6, 6-7, 10-11, 11-12, 12-13, 13-0 should be 50-feet apart.
print('\n\nPoles 3-4, 4-5, 5-6, 6-7, 10-11, 11-12, 12-13, 13-0 should be 50-feet apart.')
for (i,j) in [[3,4], [4,5], [5,6], [6,7], [10,11], [11,12], [12,13], [13,0]]:
    loc1 = list(myNodes[myNodes['id'] == i][['lat', 'lon']].values[0])
    loc2 = list(myNodes[myNodes['id'] == j][['lat', 'lon']].values[0])
    distMeters = vrv.distance2D(loc1, loc2)
    distFeet = vrv.convertDistance(distMeters, 'meters', 'feet')
    print('From Pole %d to %d: %f meters (%f feet)' % (i, j, distMeters, distFeet))

Poles 0-1, 1-2, 2-3, 7-8, 8-9, 9-10 should be 40-feet apart.
From Pole 0 to 1: 12.191304 meters (39.997716 feet)
From Pole 1 to 2: 12.192675 meters (40.002216 feet)
From Pole 2 to 3: 12.191787 meters (39.999302 feet)
From Pole 7 to 8: 12.192645 meters (40.002117 feet)
From Pole 8 to 9: 12.191770 meters (39.999244 feet)
From Pole 9 to 10: 12.191776 meters (39.999264 feet)


Poles 3-4, 4-5, 5-6, 6-7, 10-11, 11-12, 12-13, 13-0 should be 50-feet apart.
From Pole 3 to 4: 15.239125 meters (49.997128 feet)
From Pole 4 to 5: 15.240435 meters (50.001426 feet)
From Pole 5 to 6: 15.239746 meters (49.999167 feet)
From Pole 6 to 7: 15.240409 meters (50.001341 feet)
From Pole 10 to 11: 15.240449 meters (50.001474 feet)
From Pole 11 to 12: 15.239787 meters (49.999300 feet)
From Pole 12 to 13: 15.240475 meters (50.001558 feet)
From Pole 13 to 0: 15.239812 meters (49.999385 feet)
